In [28]:
from Crypto.Util import number 
with open('encrypted.bin', 'rb') as f:
    content = f.read()

So we have a linear function which is acclied to each byte

$(a*\text{val} + b) \mod N$

where $N$ is $256$ and $a$ and $b$ are chosen at random with $a$ being odd because of $gdc(a,N) = 1$

We do not have to bruteforce $a$ and $b$, because the encrycted doc is likely a cdf starting with `%cDF` as its magic number

In [29]:
len(content)

776746

In [30]:
starting_bytes = content[:4]
N = 256
c0,c1,c2,c3 = starting_bytes
c0,c1,c2,c3

(13, 112, 132, 214)

In [48]:
p0,p1,p2,p3 = [ord(x) for x in "%PDF"]
p0,p1,p2,p3

(37, 80, 68, 70)

So we get the following system of equations

$c_0 = a * p_0 + b \mod N$

$c_1 = a * p_1 + b \mod N$

$c_2 = a * p_2 + b \mod N$

$c_3 = a * p_3 + b \mod N$

We can solve this by subtracting II and I and solving III for $b$

$(c_0 - c_1) * (p_0-p_1)^{-1} = a \mod N$

$c_2 - a * p_2 = b \mod N$

After solving, we can inverse the generator:

$c = a * p + b \mod N$

$p = (c - b) * a^{-1} \mod N$

In [39]:
a = ((c0 - c1) % N * number.inverse(p0-p1,N)) % N
a

169

In [40]:
b = (c2 - a*ord("D")) % N
b

160

In [43]:
assert c3 == (a * ord("F")+ b) % N

In [46]:
a_inv = number.inverse(a,N)

with open("message.pdf","wb") as f:
    for c in content:
        f.write(
            bytes([(c - b) * a_inv % N])
        )
